# Natural Language Processing

* Las personas usan lenguajes para comunicar sus ideas.

* Las palabras son bloques de significado que tienen la ventaja de que se pueden escribir y combinar entre sí para representar una idea. 

* La primera forma de representar una idea más grande a partir de palabras es hacer manualmente una red de palabras, es decir, listar para cada palabra, las palabras que se relacionan con ella. (*good*, *goodness*, *well*, etc.). Dada una secuencia de palabras, se podría pensar en que todas las combinaciones de frases generadas de reemplazar cada palabra de la frase por sus conectadas podrían dar lugar a un significado más claro. Es más, se podría interpretar como que el significado de la frase son todas esas combinaciones juntas.

## Word2Vec

La segunda forma es con una representación distribuída. Es decir, ahora tenemos que las palabras viven en un espacio continuo y no están relacionadas por una red (lo cual hace que vivan en un espacio discreto). Esta idea hace que podamos investigar la similitud entre las palabras y sus significados. Esto depende de cómo se las entrene.

La hipótesis distribucional asume que es posible comprender el significado de las palabras a partir de su contexto. Con esto, podemos pensar en que el significado de las palabras se obtiene como consecuencia de aprender la probabilidad $P(w_{n+1}|w_1,\ldots,w_n)$ de que aparezca una palabra $w_{n+1}$ dado un conjunto de palabras $w_1,\ldots,w_n$. El algoritmo `word2vec` propone estimar esta probabilidad a partir de las muestras que aparecen en un texto.

Un poco más claro. Defino las variables $W_{c-m},W_{c-m+1},\ldots,W_c,\ldots,W_{c+m-1},W_{c+m}$ con realizaciones en un vocabulario $V= \{ w_1, \ldots, w_{|V|} \}$. El objetivo es estimar la probabilidad

$$
P(W_c=w_c|W_{c-m}=w_{c-m},\ldots,W_{c-1}=w_{c-1},W_{c+1}=w_{c+1},W_{c+m}=w_{c+m})
$$

o simplemente

$$
P(w_c|w_{c-m},\ldots,w_{c-1},w_{c+1},w_{c+m})
$$

Para eso contamos con las muestras

In [23]:
import torch
import torch.nn as nn
import numpy as np
import itertools

In [64]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, tokens_dict={}, frequencies_dict={}):
        
        self._idx_to_tk = tokens_dict
        self._tk_to_idx = {tk: idx for idx, tk in tokens_dict.items()}
        self._idx_to_freq = frequencies_dict
        self.max_idx = len(self)
        
    @classmethod
    def from_corpus(cls, corpus, cutoff_freq=0):
        corpus_words = sorted(list(set([item for sublist in corpus for item in sublist])))
        freqs_dict = {word: 0 for word in corpus_words}
        for doc in corpus:
            for token in doc:
                freqs_dict[token] += 1
        freqs = np.array(list(freqs_dict.values()))
        mask = freqs > cutoff_freq
        corpus_words = {idx: tk for idx, tk in enumerate(itertools.compress(corpus_words,mask))}
        freqs = {idx: freq for idx, freq in enumerate(freqs[mask])}
        return cls(corpus_words, freqs)

    def index_to_token(self, index):
        return self._idx_to_tk[index]

    def token_to_index(self, token):
        return self._tk_to_idx[token]
        
    def get_freq(self, tk_or_idx):
        
        if isinstance(tk_or_idx, int):
            freq = self._idx_to_freq[tk_or_idx]
        elif isinstance(tk_or_idx, str):
            freq = 0 if tk_or_idx not in self._tk_to_idx else self._idx_to_freq[self._tk_to_idx[tk_or_idx]]
        else:
            raise KeyError('{} must be either integer or string'.format(tk_or_idx))
        return freq

    def __str__(self):
        return "<Vocabulary(size={})>".format(len(self))

    def __len__(self):
        return len(self._idx_to_tk)
    
    def __getitem__(self,tk_or_idx):
        if isinstance(tk_or_idx, int):
            return self.index_to_token(tk_or_idx)
        if isinstance(tk_or_idx, str):
            return self.token_to_index(tk_or_idx)
        raise KeyError('{} must be either integer or string'.format(tk_or_idx))
        
    def __iter__(self):
        self.current = 0
        return self
    
    def __next__(self):
        if self.current >= self.max_idx:
            raise StopIteration
        else:
            token = self._idx_to_tk[self.current]
            self.current += 1
            return token

    def __contains__(self,key):
        return key in self._tk_to_idx
    
    

def samples_generator1(doc, vocabulary, window_size):
    for t, token in enumerate(doc):
        if token in vocabulary:
            len_doc = len(doc)
            cond1 = max(-1,t-window_size) == -1
            cond2 = min(t+window_size, len_doc) == len_doc
            if cond1 and cond2:
                context = itertools.chain(doc[:t],doc[t+1:])
            if cond1 and not cond2:
                context = itertools.chain(doc[:t],doc[t+1:t+window_size+1])
            if cond2 and not cond1:
                context = itertools.chain(doc[t-window_size:t],doc[t+1:])
            if not cond1 and not cond2:
                context = itertools.chain(doc[t-window_size:t],doc[t+1:t+window_size+1])

            context_list = [vocabulary.token_to_index(tk) for tk in context if tk in vocabulary]
            if len(context_list) != 0:
                yield (vocabulary.token_to_index(token), context_list)
    
    
def samples_generator2(doc, vocabulary, window_size, padding_idx):
    for t, token in enumerate(doc):
        if token in vocabulary:
            len_doc = len(doc)
            cond1 = max(-1,t-window_size) == -1
            cond2 = min(t+window_size, len_doc) == len_doc
            if cond1 and cond2:
                context = itertools.chain(doc[:t],doc[t+1:])
            if cond1 and not cond2:
                context = itertools.chain(doc[:t],doc[t+1:t+window_size+1])
            if cond2 and not cond1:
                context = itertools.chain(doc[t-window_size:t],doc[t+1:])
            if not cond1 and not cond2:
                context = itertools.chain(doc[t-window_size:t],doc[t+1:t+window_size+1])

            context_list = [vocabulary.token_to_index(tk) for tk in context if tk in vocabulary]
            yield (vocabulary.token_to_index(token), context_list)

corpus = [['w1', 'w2', 'w3', 'w4'], ['w1', 'w3', 'w3', 'w3'], ['w1'], ['w1', 'w2', 'w3', 'w4', 'w1', 'w2', 'w3', 'w4']]
cutoff_freq = 3
window_size = 2
vocabulary = Vocabulary.from_corpus(corpus,cutoff_freq=cutoff_freq)
padding_idx = len(vocabulary)

word_indeces = []
word_contexts = []
for doc in corpus:
    gen = samples_generator1(doc, vocabulary, window_size)
    for word_index, word_context in gen:
        word_indeces.append(word_index)
        padd_num = 2 * window_size - len(word_context)
        if padd_num > 0:
            word_contexts.append(word_context + [padding_idx for i in range(padd_num)])
        else:
            word_contexts.append(word_context)

word_indeces = torch.tensor(word_indeces,dtype=torch.long)
context_indeces = torch.tensor(word_contexts,dtype=torch.long)

In [21]:
vocab_size = 5
target = torch.tensor([[1,5],[1,1],[0,3]])
scores = torch.randn(3,vocab_size).view(-1,vocab_size,1).repeat(1,1,target.size(1))
lf = nn.CrossEntropyLoss(ignore_index=vocab_size,reduction='none')
loss1 = lf(scores,target)
print(loss1, loss1.mean())

loss2 = [lf(scores[:,:,0],target[:,0]), lf(scores[:,:,0],target[:,1])]
print(loss2)

lf(torch.rand(1,5),torch.tensor([5]))

tensor([[0.5702, 0.0000],
        [2.0046, 2.0046],
        [1.9594, 2.4536]]) tensor(1.4987)
[tensor([0.5702, 2.0046, 1.9594]), tensor([0.0000, 2.0046, 2.4536])]


tensor([0.])